In [31]:
import sys
sys.path.append('./models')

import torch
import onnx
import time
import threading
import netron
from tqdm import tqdm

In [32]:
print(torch.__version__)
print(onnx.__version__)

2.6.0+cu126
1.17.0


In [33]:
torch.cuda.is_available()

True

In [34]:
MODEL_PATH = "./result/best.pt" # 모델 경로
ONNX_PATH = "./onnx/panns_cnn14.onnx" # 오닉스 저장경로
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용 가능여부 확인

In [35]:
print(DEVICE)

cuda


In [36]:
# 모델 로드
model = torch.load(MODEL_PATH, weights_only=False)
model.to(DEVICE)
print(model.spectrogram_extractor.stft.pad_mode)

reflect


In [37]:
# 패딩 모드 변경
'''
STFT 입력층의 패딩의 형태를 상수의 형대로 정하는 코드
모델의 기본값은 "reflect" 이나
onnx 와의 호환성을 위해 "constant" 변경
'''
model.spectrogram_extractor.stft.pad_mode = "constant"
print(model.spectrogram_extractor.stft.pad_mode)

constant


In [38]:
# 더미 생성: (batch_size, length)
dummy_input = torch.randn(1, 16000, device=DEVICE)

In [41]:
# onnx 모델 변환

print("ONNX 변환중...")

torch.onnx.export(model, # PyTorch 모델
                  dummy_input, # 입력 텐서 (튜플도 가능)
                  ONNX_PATH, # 저장할 파일 경로
                  export_params=True, # 모델 파라미터 포함
                  opset_version=11, # onnx의 연산자 버전
                  input_names=['input'], # 입력 이름
                  output_names=['output'], # 출력 이름
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 배치 크기를 동적으로 지정
)

print(f"ONNX 변환완료'{ONNX_PATH}'에 저장됨")

ONNX 변환중...
ONNX 변환완료'./onnx/panns_cnn14.onnx'에 저장됨


In [42]:
model = onnx.load(ONNX_PATH)
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %input[FLOAT, batch_sizex16000]
) initializers (
  %spectrogram_extractor.stft.conv_real.weight[FLOAT, 257x1x512]
  %spectrogram_extractor.stft.conv_imag.weight[FLOAT, 257x1x512]
  %logmel_extractor.melW[FLOAT, 257x64]
  %bn0.weight[FLOAT, 64]
  %bn0.bias[FLOAT, 64]
  %bn0.running_mean[FLOAT, 64]
  %bn0.running_var[FLOAT, 64]
  %fc1.weight[FLOAT, 2048x2048]
  %fc1.bias[FLOAT, 2048]
  %fc_audioset.weight[FLOAT, 5x2048]
  %fc_audioset.bias[FLOAT, 5]
  %onnx::Conv_189[FLOAT, 64x1x3x3]
  %onnx::Conv_190[FLOAT, 64]
  %onnx::Conv_192[FLOAT, 64x64x3x3]
  %onnx::Conv_193[FLOAT, 64]
  %onnx::Conv_195[FLOAT, 128x64x3x3]
  %onnx::Conv_196[FLOAT, 128]
  %onnx::Conv_198[FLOAT, 128x128x3x3]
  %onnx::Conv_199[FLOAT, 128]
  %onnx::Conv_201[FLOAT, 256x128x3x3]
  %onnx::Conv_202[FLOAT, 256]
  %onnx::Conv_204[FLOAT, 256x256x3x3]
  %onnx::Conv_205[FLOAT, 256]
  %onnx::Conv_207[FLOAT, 512x256x3x3]
  %onnx::Conv_208[FLOAT, 512]
  %onnx::Conv_210[FLOAT, 512x512x3x3]
  %onnx::Conv_211[FLO

In [43]:
print("Inputs:")
for input_tensor in model.graph.input:
    print(input_tensor)

print("\nOutputs:")
for output_tensor in model.graph.output:
    print(output_tensor)

Inputs:
name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_value: 16000
      }
    }
  }
}


Outputs:
name: "output"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_value: 5
      }
    }
  }
}

name: "onnx::Gemm_185"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "batch_size"
      }
      dim {
        dim_value: 2048
      }
    }
  }
}



In [44]:
try:
    onnx.checker.check_model(model)
    print("모델이 올바른 ONNX 형식을 따릅니다.")
except onnx.checker.ValidationError as e:
    print("모델 검증 실패")
    print(e)

모델이 올바른 ONNX 형식을 따릅니다.


In [47]:
# 모델 시각화
netron.start (ONNX_PATH)

Serving './onnx/panns_cnn14.onnx' at http://localhost:8080


('localhost', 8080)